In [1]:
%pip install segmentation-models &> /dev/null
%load_ext tensorboard

import shutil
import numpy as np
import tensorflow as tf
from keras import backend as K
import segmentation_models as sm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from unet_util import dice_coef_loss, dice_coef, jacard_coef, dice_coef_loss, Residual_CNN_block, multiplication, attention_up_and_concatenate, multiplication2, attention_up_and_concatenate2, UNET_224, evaluate_prediction_result

sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [2]:
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)

# Avaiable backbones for Unet architechture
# 'vgg16' 'vgg19' 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152' 'inceptionv3' 
# 'inceptionresnetv2' 'densenet121' 'densenet169' 'densenet201' 'seresnet18' 'seresnet34' 
# 'seresnet50' 'seresnet101' 'seresnet152', and 'attentionUnet'
backend = 'densenet169'

# Source Location
# 'covington' 'rowancreek' 
source_location = 'covington'

# Target Location
# 'covington' 'rowancreek' 
target_location = 'rowancreek'

# Fine-tuning flag
# True/False
finetune = True

# Construct the model save file name
source_name = source_location+'-Unet-'+backend + ('-tf' if(finetune) else '')


# Construct the model save file name
tareget_name = source_location+'-to-'+target_location+'-Unet-'+backend

logdir = log_path + tareget_name
if(os.path.isdir(logdir)):
  shutil.rmtree(logdir)
os.makedirs(logdir, exist_ok=True)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

print('source model location: '+ model_path+source_name+'.h5')
print('target model location: '+ model_path+tareget_name+'.h5')

/content/drive/MyDrive/Transfere Learning research/Notebooks
source model location: ./models/covington-Unet-densenet169-tf.h5
target model location: ./models/covington-to-rowancreek-Unet-densenet169.h5


In [3]:
# define hyperparameters and callback modules
patience = 10
maxepoch = 500
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=patience, min_lr=1e-9, verbose=1, mode='min'),
             EarlyStopping(monitor='val_loss', patience=patience, verbose=0),
             ModelCheckpoint(model_path+tareget_name+'.h5', monitor='val_loss', save_best_only=True, verbose=0),
             TensorBoard(log_dir=logdir)]
             
# Load the best model saved by the callback module
from keras.models import load_model
if(backend != "attentionUnet"):
  model = load_model(model_path+source_name+'.h5',
                        custom_objects={'dice_coef_loss':dice_coef_loss,
                                        'dice_coef':dice_coef,})
else:
  model = load_model(model_path+source_name+'.h5',
                        custom_objects={'multiplication': multiplication,
                                        'multiplication2': multiplication2,
                                        'dice_coef_loss':dice_coef_loss, 
                                        'dice_coef':dice_coef,})

In [4]:
X_train = np.load(input_data+target_location+'/train_data.npy').astype(np.float32)
Y_train = np.load(input_data+target_location+'/train_label.npy').astype(np.float32)
X_validation = np.load(input_data+target_location+'/vali_data.npy').astype(np.float32)
Y_validation = np.load(input_data+target_location+'/vali_label.npy').astype(np.float32)

In [5]:
train_history = model.fit(x = X_train,y = Y_train, 
                          validation_data = (X_validation, Y_validation), 
                          batch_size = 16, epochs = maxepoch, verbose=0, callbacks = callbacks)


Epoch 66: ReduceLROnPlateau reducing learning rate to 6.9999998231651255e-06.


In [6]:
if(finetune):

  # For fine-tuning we need to set the tranable flag to true for the whole model
  model.trainable = True

  # Recompile the model with the smaller learning rate at the optimizer (Adam(1e-5))
  model.compile(optimizer = Adam(1e-5), loss = dice_coef_loss, metrics=[dice_coef,'accuracy'])

  # train the model again
  train_history_2 = model.fit(x = X_train, y = Y_train,
                              validation_data=(X_validation, Y_validation),
                              batch_size=16,epochs=maxepoch,
                              initial_epoch = len(train_history.history['val_loss'])-1,
                              verbose=0 ,callbacks=callbacks)


Epoch 78: ReduceLROnPlateau reducing learning rate to 6.9999998231651255e-06.


In [7]:
# Load the best model saved by the callback module
from keras.models import load_model
if(backend != "attentionUnet"):
  best_model = load_model(model_path+tareget_name+'.h5',
                        custom_objects={'dice_coef_loss':dice_coef_loss,
                                        'dice_coef':dice_coef,})
else:
  best_model = load_model(model_path+tareget_name+'.h5',
                        custom_objects={'multiplication': multiplication,
                                        'multiplication2': multiplication2,
                                        'dice_coef_loss':dice_coef_loss, 
                                        'dice_coef':dice_coef,})

# load the test data
X_test = np.load(input_data+target_location+'/bottom_half_test_data.npy').astype(np.float32)

# predict the test data using the loaded model
test_predicted= best_model.predict(X_test)

# convert the prediction probability to true or false with threshold at 0.5
test_predicted_threshold = (test_predicted > 0.5).astype(np.uint8)

# save the prediction results
np.save(prediction_path+tareget_name+'_predict.npy',test_predicted_threshold)
print('Predtion results saved: ' + prediction_path+tareget_name+'_predict.npy')

Predtion results saved: ./predicts/covington-to-rowancreek-Unet-densenet169_predict.npy


In [8]:
pred_npy = prediction_path+tareget_name+'_predict.npy'
mask_npy = input_data+target_location+'/bottom_half_test_mask.npy'
label_npy = input_data+target_location+'/bottom_half_test_label.npy'
model = model_path + tareget_name + '.h5'
text_path = prediction_path+'prediction_results.txt'

evaluate_prediction_result(target_location, pred_npy, mask_npy, label_npy, model, text_path)

Model path:./models/covington-to-rowancreek-Unet-densenet169.h5  Run at: 2022-07-19 21:53:36.008181
F1 score of Nonstream: 0.9939058363142539
F1 score of Stream: 0.7591718453166757
Precision of Nonstream: 0.9939304556843414
Precision of Stream: 0.7584294601671416
Recall of Nonstream: 0.9938812181637657
Recall of Stream: 0.7599156852507204
